In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import xarray 
import numpy as np
import logging
from netCDF4 import Dataset


hgrid_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_hgrid.nc"
topo_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_topog.nc"
ds_hgrid= xarray.open_dataset(hgrid_path)
ds_topo=xarray.open_dataset(topo_path)

print(ds_hgrid)  # Prints the variables, dimensions, and metadata
print(ds_topo)


<xarray.Dataset>
Dimensions:   (nyp: 845, nxp: 775, nx: 774, ny: 844)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 ...
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6
<xarray.Dataset>
Dimensions:  (ny: 422, nx: 387, ntiles: 1)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 ...
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1


In [2]:
original_area = ds_hgrid['area']
# Truncate dimensions to ensure even values before downsampling
nyp_truncated = 845
nxp_truncated = 773
ny_truncated = 422 * 2
nx_truncated = 386 * 2

truncated_area = original_area.isel(
    ny=slice(0, ny_truncated),
    nx=slice(0, nx_truncated)
)

# Identify the trimmed rows/columns and save their contributions
trimmed_rows_area = original_area.isel(ny=slice(ny_truncated, None), nx=slice(0, nx_truncated)).sum(dim='ny')
trimmed_cols_area = original_area.isel(ny=slice(0, ny_truncated), nx=slice(nx_truncated, None)).sum(dim='nx')
trimmed_corner_area = original_area.isel(ny=slice(ny_truncated, None), nx=slice(nx_truncated, None)).sum()

# Downsample the truncated area by taking every other point
coarsened_area = truncated_area.coarsen(ny=2, nx=2, boundary='trim').sum()

# Add the trimmed rows' areas to the bottom row of the coarsened grid
if trimmed_rows_area.size > 0:
    coarsened_area[-1, :] += trimmed_rows_area.coarsen(nx=2, boundary='trim').sum()

# Add the trimmed columns' areas to the rightmost column of the coarsened grid
if trimmed_cols_area.size > 0:
    coarsened_area[:, -1] += trimmed_cols_area.coarsen(ny=2, boundary='trim').sum()

# Add the trimmed corner area to the bottom-right cell
if trimmed_corner_area.size > 0:
    coarsened_area[-1, -1] += trimmed_corner_area
    
# Downsample the dataset
downsampled_ds_hgrid = ds_hgrid.isel(
    nyp=slice(0, nyp_truncated, 2),
    nxp=slice(0, nxp_truncated, 2),
    ny=slice(0, ny_truncated, 2),
    nx=slice(0, nx_truncated, 2)
)

downsampled_ds_hgrid['area'] = coarsened_area
# Save the downsampled dataset to a new NetCDF file
output_file='/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_3/INPUT/EXCHANGE_GRIDS/wave_hgrid.nc'
downsampled_ds_hgrid.to_netcdf(output_file)

print(f"Downsampled dataset saved to {output_file}")
original_area_sum = original_area.sum().item()
adjusted_downsampled_area_sum = coarsened_area.sum().item()
print(f"Original total area: {original_area_sum}")
print(f"Adjusted downsampled total area: {adjusted_downsampled_area_sum}")


# downsampled_ds_topo = ds_topo.isel(ny=slice(None, None, 2), nx=slice(None, None, 2))

# # Step 3: Save the downsampled data into a new NetCDF file
# output_path = '/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling_ZC/examples/NWA12.COBALT_all/INPUT/EXCHANGE_GRIDS/ocean_topo_downsampled.nc'
# downsampled_ds_topo.to_netcdf(output_path)

# print(f"Downsampled file saved to {output_path}")
# downsampled_ds_topo


ds_trimmed_topo = ds_topo.isel(
    ny=slice(0, ds_topo.dims['ny'] - (ds_topo.dims['ny'] % 2)),  # Ensure 'ny' is even
    nx=slice(0, ds_topo.dims['nx'] - (ds_topo.dims['nx'] % 2))   # Ensure 'nx' is even
)

# Downsample by selecting every other point
# downsampled_ds_topo = ds_trimmed_topo.isel(
#     ny=slice(0, None, 2),  # Take every second point along 'ny'
#     nx=slice(0, None, 2)   # Take every second point along 'nx'
# )
downsampled_ds_topo = ds_trimmed_topo.coarsen(ny=2, nx=2, boundary='trim').mean()

output_file="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_3/INPUT/EXCHANGE_GRIDS/wave_topog.nc"
# Save the downsampled dataset to a new NetCDF file
downsampled_ds_topo.to_netcdf(output_file)

print(f"Downsampled dataset saved to {output_file}")


Downsampled dataset saved to /gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_3/INPUT/EXCHANGE_GRIDS/wave_hgrid.nc
Original total area: 31735230937005.504
Adjusted downsampled total area: 31735230937005.492
Downsampled dataset saved to /gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_3/INPUT/EXCHANGE_GRIDS/wave_topog.nc


In [3]:
downsampled_ds_topo

<xarray.Dataset>
Dimensions:  (ny: 211, nx: 193, ntiles: 1)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 0.0 0.0 0.0 0.0 ... 2.95e+03 2.8e+03 2.814e+03
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1

In [4]:
downsampled_ds_hgrid

<xarray.Dataset>
Dimensions:   (nyp: 423, nxp: 387, nx: 386, ny: 422)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 3.145e+08 3.145e+08 ... 1.376e+08 3.444e+08
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6